In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D  #跟cnn有關
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD

### 1. 讀入 MNSIT 數據集

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255  #1代表只有一張矩陣(灰階) (彩色圖片有3個channel(矩陣))

In [ ]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)
* 輸入28*28的矩陣
* 做了3次covolution
* flatten(拉平)
* 做Dense
* 初初成10維向量

In [ ]:
model = Sequential()

In [ ]:
#加一個cnn的層
model.add(Conv2D(16, (3,3), padding='same',          #16個filter，大小3*3，記分板大小跟原圖一樣大
                input_shape=(28,28,1),      #告訴她input的樣子
                activation='relu'))       #激發程式的種類

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))    #加上pooling層

In [ ]:
#再加一個cnn的層
model.add(Conv2D(32, (3,3), padding='same',        #filter個數16=>32
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
#再加一個cnn的層
model.add(Conv2D(64, (3,3), padding='same',        #filter個數32=>64
                activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())    #接到標準神經網路之前要先flatten拉平

In [ ]:
model.add(Dense(54, activation='relu'))   #加上全連結的層fully connected(自己決定要幾個神經元:隨便選54)

In [ ]:
model.add(Dense(10, activation='softmax'))   #輸出10個數字:10個神經元；希望數字加起來是1:activation='softmax'

#### 看一下我們的神經網路

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

#### 組裝

In [ ]:
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### 4. step 2: fit

In [ ]:
model.fit(x_train, y_train, batch_size=100, epochs=10)  #訓練她 每做100次就檢討 epochs=10 :學10次 

Epoch 1/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0060 - accuracy: 0.9620
Epoch 2/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0051 - accuracy: 0.9673
Epoch 3/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0046 - accuracy: 0.9713
Epoch 4/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0042 - accuracy: 0.9732
Epoch 5/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0038 - accuracy: 0.9762
Epoch 6/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0036 - accuracy: 0.9775
Epoch 7/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0033 - accuracy: 0.9790
Epoch 8/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0031 - accuracy: 0.9808
Epoch 9/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.0030 - accuracy: 0.9814
Epoch 10/10
1200/1200 [==============================] - 4s 3ms/step - loss: 0.002

### Step 3. 預測

In [ ]:
result = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


### 看看測試資料表現如何

In [ ]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0052 - accuracy: 0.9674


In [ ]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 96.74%


In [ ]:
def my_predict(n):
    print('我可愛的 CNN 預測是', result[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [ ]:
from ipywidgets import interact_manual

In [ ]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [ ]:
model.save('myCNNmodel.h5')